In [1]:
import tensorflow as tf
import os, re
import random, math
import numpy as np
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings(action='ignore')
print(tf.__version__)

2.10.0


In [2]:
# 데이터 로드할 때 빠르게 로드할 수 있도록하는 설정 변수
AUTOTUNE = tf.data.experimental.AUTOTUNE
# X-RAY 이미지 사이즈 변수
IMAGE_SIZE = [180, 180]

# 데이터 경로 변수
ROOT_PATH = "C:\\Users\\박해극\\Aiffel\\AIFFEL_Quest\\mainquest4\\archive (2)"
TRAIN_PATH = ROOT_PATH + '/chest_xray/train/*/*' # *은 모든 디렉토리와 파일을 의미합니다.
VAL_PATH = ROOT_PATH + '/chest_xray/val/*/*'
TEST_PATH = ROOT_PATH + '/chest_xray/test/*/*'

# 프로젝트를 진행할 때 아래 두 변수를 변경해보세요
BATCH_SIZE = 16
EPOCHS = 10

print(ROOT_PATH)

C:\Users\박해극\Aiffel\AIFFEL_Quest\mainquest4\archive (2)


In [3]:
# 데이터 가져오기
train_filenames = tf.io.gfile.glob(TRAIN_PATH)
test_filenames = tf.io.gfile.glob(TEST_PATH)
val_filenames = tf.io.gfile.glob(VAL_PATH)

print(len(train_filenames))
print(len(test_filenames))
print(len(val_filenames))

5216
624
16


In [4]:
# train 데이터와 validation 데이터를 모두 filenames에 담습니다
filenames = tf.io.gfile.glob(TRAIN_PATH)
filenames.extend(tf.io.gfile.glob(VAL_PATH))

# 모아진 filenames를 8:2로 나눕니다
train_size = math.floor(len(filenames)*0.8)
random.seed(8)
random.shuffle(filenames)
train_filenames = filenames[:train_size]
val_filenames = filenames[train_size:]

print(len(train_filenames))
print(len(val_filenames))

4185
1047


In [5]:
print(f'Normal image path\n{filenames[0]}')
print(f'Pneumonia image path\n{filenames[2000]}')

Normal image path
C:\Users\박해극\Aiffel\AIFFEL_Quest\mainquest4\archive (2)\chest_xray\train\NORMAL\NORMAL2-IM-0707-0001.jpeg
Pneumonia image path
C:\Users\박해극\Aiffel\AIFFEL_Quest\mainquest4\archive (2)\chest_xray\train\PNEUMONIA\person610_bacteria_2475.jpeg


In [6]:
COUNT_NORMAL = len([filename for filename in train_filenames if "NORMAL" in filename])
print(f"Normal images count in training set: {COUNT_NORMAL}")


COUNT_PNEUMONIA = len([filename for filename in train_filenames if "PNEUMONIA" in filename])
print(f"Pneumonia images count in training set: {COUNT_PNEUMONIA}")

Normal images count in training set: 1072
Pneumonia images count in training set: 3113


In [7]:
train_list_ds = tf.data.Dataset.from_tensor_slices(train_filenames)
val_list_ds = tf.data.Dataset.from_tensor_slices(val_filenames)

In [8]:
TRAIN_IMG_COUNT = tf.data.experimental.cardinality(train_list_ds).numpy()
print(f"Training images count: {TRAIN_IMG_COUNT}")

VAL_IMG_COUNT = tf.data.experimental.cardinality(val_list_ds).numpy()
print(f"Validating images count: {VAL_IMG_COUNT}")

Training images count: 4185
Validating images count: 1047


In [9]:
# 파일 경로의 끝에서 두번째 부분을 확인하면 양성과 음성을 구분할 수 있습니다
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2] == "PNEUMONIA"   # 폐렴이면 양성(True), 노말이면 음성(False)

In [10]:
# 이미지를 알맞은 형식으로 바꿉니다.
def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3) # 이미지를 uint8 tensor로 수정
    img = tf.image.convert_image_dtype(img, tf.float32) # float32 타입으로 수정
    img = tf.image.resize(img, IMAGE_SIZE) # 이미지 사이즈를 IMAGE_SIZE로 수정
    return img

# 이미지 파일의 경로를 입력하면 이미지와 라벨을 읽어옵니다.
def process_path(file_path):
    label = get_label(file_path) # 라벨 검출
    img = tf.io.read_file(file_path) # 이미지 읽기
    img = decode_img(img) # 이미지를 알맞은 형식으로 수정
    return img, label

In [11]:
train_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [12]:
# 이미지 shape 확인
for image, label in train_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (180, 180, 3)
Label:  False


In [13]:
# test 이미지 수 카운트
test_list_ds = tf.data.Dataset.list_files(TEST_PATH)
TEST_IMAGE_COUNT = tf.data.experimental.cardinality(test_list_ds).numpy()
test_ds = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE)

print(TEST_IMAGE_COUNT)

624


# Data Augmentation

In [14]:
def augment(image,label):
    image = tf.image.random_flip_left_right(image)  # 랜덤하게 좌우를 반전합니다.
    return image,label

def prepare_for_training(ds, shuffle_buffer_size=1000):
    # augment 적용 부분이 배치처리 함수에 추가되었습니다.
    ds = ds.map(
            augment,       # augment 함수 적용
            num_parallel_calls=2
        )
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat()
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

train_ds = prepare_for_training(train_ds)
val_ds = prepare_for_training(val_ds)

## 데이터 시각화

In [ ]:
# 이미지 배치를 입력하면 여러장의 이미지를 보여줍니다.
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(BATCH_SIZE):
        ax = plt.subplot(4,math.ceil(BATCH_SIZE/4),n+1)
        plt.imshow(image_batch[n])
        if label_batch[n]:
            plt.title("PNEUMONIA")
        else:
            plt.title("NORMAL")
        plt.axis("off")


image_batch, label_batch = next(iter(train_ds))
show_batch(image_batch.numpy(), label_batch.numpy())

## 모델 쌓기 시작

In [ ]:
# Convolution Block 만들기
def conv_block(filters):
    block = tf.keras.Sequential([
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D()
    ])
    
    return block

In [ ]:
# Dense Block 만들기
def dense_block(units, dropout_rate):
    block = tf.keras.Sequential([
        tf.keras.layers.Dense(units, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate)
    ])
    
    return block

In [ ]:
# 모델 쌓기
from tensorflow.keras.layers import UpSampling2D, AveragePooling2D
from tensorflow.keras.models import Model

def build_model():
    # Conv1 레이어
    input_layer = layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    conv1 = layers.Conv2D(filters=64, kernel_size=(7, 7), strides=2, activation='relu', padding='same')(input_layer)
    maxpool_1 = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv1)

    # Conv2-1 레이어
    conv2 = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=2, padding='same')(maxpool_1)
    batchnorm_01 = layers.BatchNormalization()(conv2)
    activation_01 = layers.Activation('relu')(batchnorm_01)
    conv3 = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=2, padding='same')(activation_01)
    batchnorm_02 = layers.BatchNormalization()(conv3)
    upsampled_batchnorm_02 = UpSampling2D(size=(4, 4))(batchnorm_02)
    concat01 = layers.concatenate([upsampled_batchnorm_02, maxpool_1], axis=-1)
    activation_02 = layers.Activation('relu')(concat01)
        
    # Conv2-2 레이어
    conv4 = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=2, padding='same')(activation_02)
    batchnorm_03 = layers.BatchNormalization()(conv4)
    activation_03 = layers.Activation('relu')(batchnorm_03)
    conv5 = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=2, padding='same')(activation_03)
    batchnorm_04 = layers.BatchNormalization()(conv5)
    upsampled_batchnorm_04 = UpSampling2D(size=(4, 4))(batchnorm_04)
    concat02 = layers.concatenate([upsampled_batchnorm_04, activation_02], axis=-1)
    activation_04 = layers.Activation('relu')(concat02)

    # Conv3-1 레이어
    conv6 = layers.Conv2D(filters=128, kernel_size=(3, 3), strides=2, padding='same')(activation_04)
    batchnorm_05 = layers.BatchNormalization()(conv6)
    activation_05 = layers.Activation('relu')(batchnorm_05)
    conv7 = layers.Conv2D(filters=128, kernel_size=(3, 3), strides=2, padding='same')(activation_05)
    batchnorm_06 = layers.BatchNormalization()(conv7)
    upsampled_batchnorm_06 = UpSampling2D(size=(4, 4))(batchnorm_06)
    concat03 = layers.concatenate([upsampled_batchnorm_06, activation_04], axis=-1)
    activation_06 = layers.Activation('relu')(concat03)

    # Conv3-2 레이어
    conv8 = layers.Conv2D(filters=128, kernel_size=(3, 3), strides=2, padding='same')(activation_06)
    batchnorm_07 = layers.BatchNormalization()(conv8)
    activation_07 = layers.Activation('relu')(batchnorm_07)
    conv9 = layers.Conv2D(filters=128, kernel_size=(3, 3), strides=2, padding='same')(activation_07)
    batchnorm_08 = layers.BatchNormalization()(conv9)
    upsampled_batchnorm_08 = UpSampling2D(size=(4, 4))(batchnorm_08)
    concat04 = layers.concatenate([upsampled_batchnorm_08, activation_06], axis=-1)
    activation_08 = layers.Activation('relu')(concat04)

    # Conv 4-1 레이어
    conv10 = layers.Conv2D(filters=256, kernel_size=(3, 3), strides=2, padding='same')(activation_08)
    batchnorm_09 = layers.BatchNormalization()(conv10)
    activation_09 = layers.Activation('relu')(batchnorm_09)
    conv11 = layers.Conv2D(filters=256, kernel_size=(3, 3), strides=2, padding='same')(activation_09)
    batchnorm_10 = layers.BatchNormalization()(conv11)
    upsampled_batchnorm_10 = UpSampling2D(size=(4, 4))(batchnorm_10)
    concat05 = layers.concatenate([upsampled_batchnorm_10, activation_08], axis=-1)
    activation_10 = layers.Activation('relu')(concat05)

    # Conv 4-2 레이어
    conv12 = layers.Conv2D(filters=256, kernel_size=(3, 3), strides=2, padding='same')(activation_10)
    batchnorm_11 = layers.BatchNormalization()(conv12)
    activation_11 = layers.Activation('relu')(batchnorm_11)
    conv13 = layers.Conv2D(filters=256, kernel_size=(3, 3), strides=2, padding='same')(activation_11)
    batchnorm_12 = layers.BatchNormalization()(conv13)
    upsampled_batchnorm_12 = UpSampling2D(size=(4, 4))(batchnorm_12)
    concat06 = layers.concatenate([upsampled_batchnorm_12, activation_10], axis=-1)
    activation_12 = layers.Activation('relu')(concat06)

    # Conv 5-1 레이어
    conv14 = layers.Conv2D(filters=512, kernel_size=(3, 3), strides=2, padding='same')(activation_12)
    batchnorm_13 = layers.BatchNormalization()(conv14)
    activation_13 = layers.Activation('relu')(batchnorm_13)
    conv15 = layers.Conv2D(filters=512, kernel_size=(3, 3), strides=2, padding='same')(activation_13)
    batchnorm_14 = layers.BatchNormalization()(conv15)
    upsampled_batchnorm_14 = UpSampling2D(size=(4, 4))(batchnorm_14)
    concat07 = layers.concatenate([upsampled_batchnorm_14, activation_12], axis=-1)
    activation_14 = layers.Activation('relu')(concat07)

    # Conv 5-2 레이어
    conv16 = layers.Conv2D(filters=512, kernel_size=(3, 3), strides=2, padding='same')(activation_14)
    batchnorm_15 = layers.BatchNormalization()(conv16)
    activation_15 = layers.Activation('relu')(batchnorm_15)
    conv17 = layers.Conv2D(filters=512, kernel_size=(3, 3), strides=2, padding='same')(activation_15)
    batchnorm_16 = layers.BatchNormalization()(conv17)
    upsampled_batchnorm_16 = UpSampling2D(size=(4, 4))(batchnorm_16)
    concat08 = layers.concatenate([upsampled_batchnorm_16, activation_14], axis=-1)
    activation_16 = layers.Activation('relu')(concat08)

    # Average Pooling
    avg_pool = layers.AveragePooling2D(pool_size=(7, 7), strides=(1, 1))(activation_16)

    # Dense 레이어
    flatten_layer = layers.Flatten()(avg_pool)
    Affine1 = layers.Dense(100)(flatten_layer)
    batchnorm_17 = layers.BatchNormalization()(Affine1)
    activation_17 = layers.Activation('relu')(batchnorm_17)

    # output 레이어
    output_layer = layers.Dense(1, activation='sigmoid')(activation_17)

    # 모델 정의
    model = Model(inputs=input_layer, outputs=output_layer)
    
    
    return model

## 데이터 imbalance 처리

In [ ]:
# Normal 데이터와 페렴데이터 수 비율 보기
weight_for_0 = (1 / COUNT_NORMAL)*(TRAIN_IMG_COUNT)/2.0 
weight_for_1 = (1 / COUNT_PNEUMONIA)*(TRAIN_IMG_COUNT)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for NORMAL: {:.2f}'.format(weight_for_0))
print('Weight for PNEUMONIA: {:.2f}'.format(weight_for_1))

## 모델 훈련

In [ ]:
# GPU 선택하기
import tensorflow.keras.layers as layers
from tensorflow.keras import backend as K

K.clear_session()

with tf.device('/GPU:0'):
    model = build_model()

    METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=METRICS
    )

model.summary()

In [ ]:
from tensorflow.keras import backend as K
K.clear_session()
with tf.device('/GPU:0'):
    history = model.fit(
        train_ds,
        steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=val_ds,
        validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
        class_weight=class_weight,
    )

In [ ]:
# 결과 확인
fig, ax = plt.subplots(1, 4, figsize=(20, 3))
ax = ax.ravel()

for i, met in enumerate(['precision', 'recall', 'accuracy', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

In [ ]:
# 모델 테스트
loss, accuracy, precision, recall = model.evaluate(test_ds)
print(f'Loss: {loss},\nAccuracy: {accuracy},\nPrecision: {precision},\nRecall: {recall}')